### Import the required libraries

In [22]:
from dotenv import load_dotenv
load_dotenv()
import os 
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec
from os.path import join, dirname
import json

### Make a connection with the Gemini API

In [18]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

### Make connection with Pinecone

In [19]:
pc = Pinecone(api_key= os.getenv("PINECONE_API_KEY"))


In [21]:
pc.create_index(
    name='rag',
    dimension=768,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

In [23]:
data = json.load(open('reviews.json'))

In [26]:
data

{'reviews': [{'professor': 'Dr. John Doe',
   'subject': 'Physics',
   'stars': 4,
   'review': 'Engaging lectures and clear explanations. Sometimes a bit fast-paced.'},
  {'professor': 'Dr. Jane Smith',
   'subject': 'Mathematics',
   'stars': 5,
   'review': 'Incredibly knowledgeable and helpful during office hours.'},
  {'professor': 'Dr. Alan Brown',
   'subject': 'Chemistry',
   'stars': 3,
   'review': 'Challenging exams but fair grading. Lectures can be dry.'},
  {'professor': 'Dr. Emma Davis',
   'subject': 'Biology',
   'stars': 4,
   'review': 'Interesting course with plenty of hands-on activities.'},
  {'professor': 'Dr. Michael Johnson',
   'subject': 'History',
   'stars': 2,
   'review': 'Lectures are monotonous, but the material is important.'},
  {'professor': 'Dr. Linda Wilson',
   'subject': 'English',
   'stars': 5,
   'review': 'Passionate about literature, makes every class engaging.'},
  {'professor': 'Dr. Robert Lee',
   'subject': 'Computer Science',
   'stars':

In [28]:
processed_data = []
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

In [43]:
for reviews in data['reviews']: 
    result = genai.embed_content(
        model="models/text-embedding-004",
        content= reviews['review'],
        task_type="retrieval_document",
    )
    processed_data.append({
        "values":result['embedding'],
        "id":reviews['professor'],
        "metadata":{
            "review":reviews["review"],
            "subject":reviews["subject"],
            "stars":reviews["stars"]
        }
    })


In [47]:
index = pc.Index('rag')
index.upsert(
    vectors= processed_data,
    namespace='ns1',
    
)

{'upserted_count': 20}

In [48]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}